In [11]:
# -*- coding: utf-8 -*-
"""
Consolidate five workbooks into 52 combined scenarios and produce:
1) Provincial annual sums
2) National annual sums
3) 2020–2030 totals (national)
4) Province-level Δ vs BS+BS (2020–2030)
5) National Δ vs BS+BS (2020–2030)
6) (Optional) Five city-level merged sheets with exact headers

Place this script in the same folder as the 5 Excel files or edit BASE_DIR.
"""

from pathlib import Path
from collections import defaultdict
from openpyxl import load_workbook
import pandas as pd

# ========= USER SETTINGS =========
BASE_DIR = Path("./output data")   # <- adjust if needed
OUTFILE  = BASE_DIR / "Consolidated_52_scenarios_environment with transport_metal.xlsx"
WRITE_CITY_LEVEL = False   # True = also write the 5 big city-level merged sheets

FILES = {
    # family : (filename, [sheet names], demand-col-name inside that workbook)
    "AR": ("Environmental impact and metal recovery results under AR scenario.xlsx",
           ["AR1 scenario","AR2 scenario","AR3 scenario"], "AR Scenario"),
    "BS": ("Environmental impact and metal recovery results under BS scenario.xlsx",
           ["Baseline scenario"], None),
    "ES": ("Environmental impact and metal recovery results under ES scenario.xlsx",
           ["ES1 scenario","ES2 scenario","ES3 scenario"], "ES"),
    "SU": ("Environmental impact and metal recovery results under SU scenario.xlsx",
           ["SU1 scenario","SU2 scenario","SU3 scenario"], "SU Scenario"),
    "TO": ("Environmental impact and metal recovery results under TO scenario.xlsx",
           ["TO1 scenario","TO2 scenario","TO3 scenario"], "TO Scenario"),
}

NUM_COLS = [
    "Abiotic depletion","Abiotic depletion (fossil fuels)","Acidification","Eutrophication",
    "Fresh water aquatic ecotox.","Global warming (GWP100a)","Human toxicity",
    "Marine aquatic ecotoxicity","Ozone layer depletion (ODP)","Photochemical oxidation",
    "Terrestrial ecotoxicity","lithium","nickel","cobalt","manganese"
]

# ---- optional city-sheet column orders (exactly as you gave) ----
CITY_COLS = {
    "SU": ["SU Scenario","Scenario","Year","Province","City","Code","Battery type","SU Ratio"] + NUM_COLS,
    "AR": ["AR Scenario","Scenario","Year","Province","City","Code","Battery type","AR Ratio"] + NUM_COLS,
    "ES": ["ES","Scenario","Year","Province","City","Code","Battery type"] + NUM_COLS,
    "BS": ["Scenario","Year","Province","City","Code","Battery type"] + NUM_COLS,
    "TO": ["TO Scenario","Scenario","Year","Province","City","Code","Battery type"] + NUM_COLS,
}

def _add_vec(acc, vals):
    for i, v in enumerate(vals):
        if v is None or v == "":
            continue
        acc[i] += float(v)

# ---- streaming aggregation (fast & memory-light) ----
prov_annual = defaultdict(lambda: [0.0]*len(NUM_COLS))
nat_annual  = defaultdict(lambda: [0.0]*len(NUM_COLS))
totals_nat  = defaultdict(lambda: [0.0]*len(NUM_COLS))
prov_total  = defaultdict(lambda: [0.0]*len(NUM_COLS))
bsbs_prov   = defaultdict(lambda: [0.0]*len(NUM_COLS))
bsbs_nat    = [0.0]*len(NUM_COLS)

for fam, (fname, sheets, fam_col) in FILES.items():
    wb = load_workbook(BASE_DIR / fname, read_only=True, data_only=True)
    for s in sheets:
        ws = wb[s]
        header = list(next(ws.iter_rows(min_row=1, max_row=1, values_only=True)))
        col_idx = {name: i for i, name in enumerate(header)}

        # Required columns present?
        req = ["Scenario","Year","Province"] + ([fam_col] if fam_col else [])
        missing = [c for c in req if c not in col_idx]
        if missing:
            raise RuntimeError(f"Missing columns {missing} in {fname} / {s}")

        idx_supp = col_idx["Scenario"]
        idx_year = col_idx["Year"]
        idx_prov = col_idx["Province"]
        idx_demd = col_idx.get(fam_col) if fam_col else None
        idx_nums = [col_idx.get(n) for n in NUM_COLS]   # may all exist, as designed

        for row in ws.iter_rows(min_row=2, values_only=True):
            sup = row[idx_supp]
            yr  = row[idx_year]
            prv = row[idx_prov]
            if sup is None or yr is None or prv is None:
                continue
            sup = str(sup)
            year = int(yr)
            prov = str(prv)
            demd = "BS" if fam == "BS" else str(row[idx_demd])
            combined = f"{sup}+{demd}"
            vals = [row[i] if i is not None else None for i in idx_nums]

            _add_vec(prov_annual[(combined, sup, demd, year, prov)], vals)
            _add_vec(nat_annual[(combined, sup, demd, year)], vals)
            _add_vec(totals_nat[(combined, sup, demd)], vals)
            _add_vec(prov_total[(combined, sup, demd, prov)], vals)

            # baseline accumulation: BS supply + BS demand inside BS workbook
            if fam == "BS" and sup == "BS":
                _add_vec(bsbs_prov[(prov,)], vals)
                _add_vec(bsbs_nat, vals)

# ---- dicts -> DataFrames ----
def dict_to_df(dct, key_cols):
    rows = [list(k)+v for k, v in dct.items()]
    return pd.DataFrame(rows, columns=key_cols + NUM_COLS)

df_prov_annual = dict_to_df(prov_annual, ["Combined scenario","Scenario","Demand scenario","Year","Province"])
df_nat_annual  = dict_to_df(nat_annual,  ["Combined scenario","Scenario","Demand scenario","Year"])
df_totals_nat  = dict_to_df(totals_nat,  ["Combined scenario","Scenario","Demand scenario"])
df_prov_total  = dict_to_df(prov_total,  ["Combined scenario","Scenario","Demand scenario","Province"])
df_bsbs_prov   = dict_to_df(bsbs_prov,   ["Province"])

# ---- build Δ vs BS+BS (provincial & national) ----
prov_diff = df_prov_total.merge(
    df_bsbs_prov.add_suffix(" (BS+BS)"),
    left_on="Province", right_on="Province (BS+BS)",
    how="left"
)
for c in NUM_COLS:
    prov_diff[f"{c} (Δ vs BS+BS)"] = prov_diff[c] - prov_diff[f"{c} (BS+BS)"]
prov_diff_out = prov_diff[
    ["Combined scenario","Scenario","Demand scenario","Province"]
    + [f"{c} (Δ vs BS+BS)" for c in NUM_COLS]
]

bsbs_nat_row = pd.Series(bsbs_nat, index=NUM_COLS)
nat_diff_out = df_totals_nat.copy()
for c in NUM_COLS:
    nat_diff_out[f"{c} (Δ vs BS+BS)"] = nat_diff_out[c] - bsbs_nat_row[c]

# ---- (optional) city-level merged sheets with exact headers ----
city_frames = {}
if WRITE_CITY_LEVEL:
    for fam, (fname, sheets, fam_col) in FILES.items():
        fam_frames = []
        for s in sheets:
            df = pd.read_excel(BASE_DIR / fname, sheet_name=s)
            # Order columns to the requested header if available; otherwise keep intersect
            wanted = CITY_COLS["BS"]  # default
            if fam in CITY_COLS: wanted = CITY_COLS[fam]
            cols = [c for c in wanted if c in df.columns]
            df = df[cols].copy()
            fam_frames.append(df)
        city_frames[fam] = pd.concat(fam_frames, ignore_index=True)

# ---- write everything ----
with pd.ExcelWriter(OUTFILE, engine="openpyxl") as w:
    df_prov_annual.to_excel(w, sheet_name="Provincial annual sums", index=False)
    df_nat_annual.to_excel(w,  sheet_name="National annual sums",   index=False)
    df_totals_nat.to_excel(w,  sheet_name="2020–2030 totals (nat.)", index=False)
    prov_diff_out.to_excel(w,  sheet_name="Δ 2020–2030 vs BS+BS (prov)", index=False)
    nat_diff_out[["Combined scenario","Scenario","Demand scenario"]
                 + [f"{c} (Δ vs BS+BS)" for c in NUM_COLS]].to_excel(
        w, sheet_name="Δ 2020–2030 vs BS+BS (nat)", index=False
    )
    if WRITE_CITY_LEVEL:
        # exact headers per your request
        if "BS" in city_frames: city_frames["BS"].to_excel(w, sheet_name="BS – merged (city)", index=False)
        if "AR" in city_frames: city_frames["AR"].to_excel(w, sheet_name="AR – merged (city)", index=False)
        if "ES" in city_frames: city_frames["ES"].to_excel(w, sheet_name="ES – merged (city)", index=False)
        if "SU" in city_frames: city_frames["SU"].to_excel(w, sheet_name="SU – merged (city)", index=False)
        if "TO" in city_frames: city_frames["TO"].to_excel(w, sheet_name="TO – merged (city)", index=False)

print(f"Done: {OUTFILE}")


Done: output data\Consolidated_52_scenarios_environment_metal.xlsx


In [15]:
# === Fixed code ===
import numpy as np
import pandas as pd
from pathlib import Path

# Environmental and metal columns (consistent with main script)
ENV_COLS = [
    "Abiotic depletion","Abiotic depletion (fossil fuels)","Acidification","Eutrophication",
    "Fresh water aquatic ecotox.","Global warming (GWP100a)","Human toxicity",
    "Marine aquatic ecotoxicity","Ozone layer depletion (ODP)","Photochemical oxidation","Terrestrial ecotoxicity"
]
METAL_COLS = ["lithium","nickel","cobalt","manganese"]
NUM_COLS = ENV_COLS + METAL_COLS

# Define output file path
BASE_DIR = Path("./output data")  # Adjust according to actual situation
OUTFILE = BASE_DIR / "Consolidated_52_scenarios_environment with transport_metal.xlsx"

# Ensure output directory exists
BASE_DIR.mkdir(parents=True, exist_ok=True)

def sum_with_transport(df: pd.DataFrame, env_cols=ENV_COLS):
    """
    Add processing stage + transport stage column by column:
    For each environmental column c:
      If 'c (transport)' column exists, then total[c] = df[c] + df['c (transport)'];
      Otherwise total[c] = df[c] (or 0 if doesn't exist).
    Metal columns don't add transport, keep original values.
    Returns: DataFrame after addition (removing * (transport) columns)
    """
    out = df.copy()
    # Ensure base columns exist (fill with 0 if not)
    for c in env_cols:
        if c not in out.columns:
            out[c] = 0.0
        base = pd.to_numeric(out[c], errors="coerce").fillna(0.0)
        tcol = f"{c} (transport)"
        if tcol in out.columns:
            trans = pd.to_numeric(out[tcol], errors="coerce").fillna(0.0)
            out[c] = base + trans
        else:
            out[c] = base
    # Metal columns keep original values (fill with 0 if not exist)
    for c in METAL_COLS:
        if c not in out.columns:
            out[c] = 0.0
        out[c] = pd.to_numeric(out[c], errors="coerce").fillna(0.0)
    # Remove transport intermediate columns
    drop_cols = [c for c in out.columns if c.endswith(" (transport)")]
    if drop_cols:
        out = out.drop(columns=drop_cols)
    return out

# ---- 1) Provincial annual (with transport) ----
# Re-merge once here to ensure getting '(transport)' columns
_key_prov_yr = ["Scenario","Year","Province"]
left = df_prov_annual.merge(
    trans_prov_annual, on=_key_prov_yr, how="left", suffixes=(""," (transport)")
).fillna(0.0)
df_prov_annual_transport = sum_with_transport(left, ENV_COLS)

# ---- 2) National annual (with transport) ----
_key_nat_yr = ["Scenario","Year"]
left = df_nat_annual.merge(
    trans_nat_annual, on=_key_nat_yr, how="left", suffixes=(""," (transport)")
).fillna(0.0)
df_nat_annual_transport = sum_with_transport(left, ENV_COLS)

# ---- 3) Provincial 2020–2030 total (with transport) ----
_key_prov = ["Scenario","Province"]
left = df_prov_total.merge(
    trans_prov_total, on=_key_prov, how="left", suffixes=(""," (transport)")
).fillna(0.0)
df_prov_total_transport = sum_with_transport(left, ENV_COLS)

# ---- 4) National 2020–2030 total (with transport) ----
left = df_totals_nat.merge(
    trans_nat_total, on=["Scenario"], how="left", suffixes=(""," (transport)")
).fillna(0.0)
df_totals_nat_transport = sum_with_transport(left, ENV_COLS)

# ---- 5) Rebuild BS+BS (with transport) as baseline ----
# First add BS transport totals to provincial/national baseline
bs_trans_prov = trans_prov_total[trans_prov_total["Scenario"]=="BS"].drop(columns=["Scenario"]).copy()
bs_trans_nat  = trans_nat_total[trans_nat_total["Scenario"]=="BS"].drop(columns=["Scenario"]).copy()

df_bsbs_prov_env = df_bsbs_prov.copy()
for c in ENV_COLS:
    tc = bs_trans_prov.set_index("Province")[c]
    df_bsbs_prov_env[c] = pd.to_numeric(df_bsbs_prov_env[c], errors="coerce").fillna(0.0) + df_bsbs_prov_env["Province"].map(tc).fillna(0.0)

bsbs_nat_with_trans = bsbs_nat_row.copy()
for c in ENV_COLS:
    add_val = float(bs_trans_nat[c].iloc[0]) if not bs_trans_nat.empty else 0.0
    base_val = pd.to_numeric(bsbs_nat_with_trans[c], errors="coerce")
    if pd.isna(base_val):
        base_val = 0.0
    bsbs_nat_with_trans[c] = float(base_val) + add_val

# ---- 6) Calculate Δ & Δ% (with transport), including metal change ratios ----
def delta_and_ratio(df_vals: pd.DataFrame, df_bs_ref: pd.DataFrame, key_cols):
    merged = df_vals.merge(
        df_bs_ref.add_suffix(" (BS+BS)"),
        left_on=key_cols, right_on=[k+" (BS+BS)" for k in key_cols],
        how="left"
    )
    out = merged[key_cols + NUM_COLS].copy()

    # Δ
    for c in NUM_COLS:
        out[f"{c} (Δ vs BS+BS) (+transport)"] = pd.to_numeric(out[c], errors="coerce").fillna(0.0) - \
                                                pd.to_numeric(merged[f"{c} (BS+BS)"], errors="coerce").fillna(0.0)
    # Δ% environmental
    for c in ENV_COLS:
        base = pd.to_numeric(merged[f"{c} (BS+BS)"], errors="coerce")
        base = base.replace(0, np.nan)
        out[f"{c} (Δ% vs BS+BS) (+transport)"] = (out[f"{c} (Δ vs BS+BS) (+transport)"] / base).replace([np.inf,-np.inf], np.nan).fillna(0.0)
    # Metal Δ%
    for c in METAL_COLS:
        base = pd.to_numeric(merged[f"{c} (BS+BS)"], errors="coerce")
        base = base.replace(0, np.nan)
        out[f"{c} (Δ% vs BS+BS) (+transport)"] = (out[f"{c} (Δ vs BS+BS) (+transport)"] / base).replace([np.inf,-np.inf], np.nan).fillna(0.0)

    return out

# Provincial baseline reference (create DF with placeholder keys from df_bsbs_prov_env)
bs_ref_prov_df = pd.DataFrame({
    "Combined scenario": ["BS+BS"]*len(df_bsbs_prov_env),
    "Scenario": ["BS"]*len(df_bsbs_prov_env),
    "Demand scenario": ["BS"]*len(df_bsbs_prov_env),
    "Province": df_bsbs_prov_env["Province"]
})
for c in NUM_COLS:
    bs_ref_prov_df[c] = pd.to_numeric(df_bsbs_prov_env[c], errors="coerce").fillna(0.0).values

prov_keys = ["Combined scenario","Scenario","Demand scenario","Province"]
# Ensure all key columns have consistent data types
key_columns = ["Combined scenario", "Scenario", "Demand scenario", "Province"]

for col in key_columns:
    if col in df_prov_total_transport.columns:
        df_prov_total_transport[col] = df_prov_total_transport[col].astype(str)
    if col in bs_ref_prov_df.columns:
        bs_ref_prov_df[col] = bs_ref_prov_df[col].astype(str)

# Execute provincial delta_and_ratio function (call only once)
prov_delta_ratio_transport = delta_and_ratio(
    df_vals=df_prov_total_transport[prov_keys + NUM_COLS].copy(),
    df_bs_ref=bs_ref_prov_df[["Combined scenario", "Scenario", "Demand scenario", "Province"] + NUM_COLS].copy(),
    key_cols=prov_keys
)

# National baseline reference (Series -> DF)
bs_ref_nat = pd.DataFrame([["BS+BS","BS","BS"] + list(pd.to_numeric(bsbs_nat_with_trans[NUM_COLS], errors="coerce").fillna(0.0).values)],
                          columns=["Combined scenario","Scenario","Demand scenario"] + NUM_COLS)

# Ensure national key columns have consistent data types
nat_key_columns = ["Combined scenario", "Scenario", "Demand scenario"]

for col in nat_key_columns:
    if col in df_totals_nat_transport.columns:
        df_totals_nat_transport[col] = df_totals_nat_transport[col].astype(str)
    if col in bs_ref_nat.columns:
        bs_ref_nat[col] = bs_ref_nat[col].astype(str)

# Execute national delta_and_ratio function (call only once)
nat_delta_ratio_transport = delta_and_ratio(
    df_vals=df_totals_nat_transport[nat_key_columns + NUM_COLS].copy(),
    df_bs_ref=bs_ref_nat[nat_key_columns + NUM_COLS].copy(),
    key_cols=nat_key_columns
)

# ---- 7) Write results to same OUTFILE (append/overwrite same sheets) ----
try:
    with pd.ExcelWriter(OUTFILE, engine="openpyxl", mode="a", if_sheet_exists="replace") as w:
        df_prov_annual_transport.to_excel(w, sheet_name="Provincial annual sums (+transport)", index=False)
        df_nat_annual_transport.to_excel(w, sheet_name="National annual sums (+transport)", index=False)
        df_prov_total_transport.to_excel(w, sheet_name="2020–2030 totals (prov., +transport)", index=False)
        df_totals_nat_transport.to_excel(w, sheet_name="2020–2030 totals (nat., +transport)", index=False)
        prov_delta_ratio_transport.to_excel(w, sheet_name="Δ 2020–2030 vs BS+BS (prov, +transport+ratio)", index=False)
        nat_delta_ratio_transport.to_excel(w, sheet_name="Δ 2020–2030 vs BS+BS (nat, +transport+ratio)", index=False)
    
    print("✅ Fixed: transport merged and Δ/ratios recomputed without KeyError.")
except FileNotFoundError:
    # If file doesn't exist, create new file
    with pd.ExcelWriter(OUTFILE, engine="openpyxl") as w:
        df_prov_annual_transport.to_excel(w, sheet_name="Provincial annual sums (+transport)", index=False)
        df_nat_annual_transport.to_excel(w, sheet_name="National annual sums (+transport)", index=False)
        df_prov_total_transport.to_excel(w, sheet_name="2020–2030 totals (prov., +transport)", index=False)
        df_totals_nat_transport.to_excel(w, sheet_name="2020–2030 totals (nat., +transport)", index=False)
        prov_delta_ratio_transport.to_excel(w, sheet_name="Δ 2020–2030 vs BS+BS (prov, +transport+ratio)", index=False)
        nat_delta_ratio_transport.to_excel(w, sheet_name="Δ 2020–2030 vs BS+BS (nat, +transport+ratio)", index=False)
    
    print("✅ Fixed: transport merged and Δ/ratios recomputed. Created new file.")

C:\Users\pengf\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


✅ Fixed: transport merged and Δ/ratios recomputed without KeyError.
